### Built-In Sequences

##### Container Sequences
 `list`, `tuple` and `collections.deque` can hold items of different types
    
##### Flat Sequences
 `str`, `bytearray`, `memoryview` and `array.array` can hold items of one type.

##### Mutable Sequences
 `list`, `bytearray`, `array.array`, `collections.deque` and `memoryview`

##### Immutable Sequences
 `tuple`, `str` and `bytes`

![title](assets/chp2-umldiagram.png)

### List Comprehensions and Generator Expressions

A quick way to build a sequence is using list comprehensions (if the target is a list) or a generator expression (for all other kinds of sequences). This makes code more readable and ofetn daster at the same time.

Let's build a list without using list comprehension

In [1]:
symbols = '$¢£¥€¤'
codes = []
for symbol in symbols:
    codes.append(ord(symbol)) #ord gets the unicode point 
                              #for one character of a string
codes

[36, 162, 163, 165, 8364, 164]

Now let's try using list comprehension

In [2]:
symbols = '$¢£¥€¤'
codes = [ord(symbol) for symbol in symbols]
codes

[36, 162, 163, 165, 8364, 164]

Using listcomps make the code easier to read. A for loop is used do lots of different things: scanning a sequence to count or pick items, computing aggregates (sums, averages), or any number of other processing tasks.

But a listcomp is meant to do only one thing, to build a new list. 
If you are not doing something with the produced list, you should not use that
syntax. Also, try to keep it short. If the list comprehension spans more than two lines, it is probably best to break it apart or rewrite as a plain old for loop.

### Listcomps Versus map and filter
Listcomps do everything the `map` and `filter` functions do, without the contortions of the functionally challenged Python `lambda`. 

Let's see a list built using listcomp

In [4]:
symbols = '$¢£¥€¤'
beyond_ascii = [ord(s) for s in symbols if ord(s) > 127]
beyond_ascii

[162, 163, 165, 8364, 164]

Now let's build a list using map/filter

In [6]:
beyond_ascii = list(filter(lambda c: c > 127, map(ord,symbols)))
beyond_ascii

[162, 163, 165, 8364, 164]

It is generally believed that listcomps are often slower than using map and filter equivalents but that's not the case. Also, using map and filter affects readability. The below python script shows the speed comparison.

In [7]:
import timeit

TIMES = 10000

SETUP = """
symbols = '$¢£¥€¤'
def non_ascii(c):
    return c > 127
"""

def clock(label, cmd):
    res = timeit.repeat(cmd, setup=SETUP, number=TIMES)
    print(label, *('{:.3f}'.format(x) for x in res))

clock('listcomp        :', '[ord(s) for s in symbols if ord(s) > 127]')
clock('listcomp + func :', '[ord(s) for s in symbols if non_ascii(ord(s))]')
clock('filter + lambda :', 'list(filter(lambda c: c > 127, map(ord, symbols)))')
clock('filter + func   :', 'list(filter(non_ascii, map(ord, symbols)))')

listcomp        : 0.012 0.012 0.013 0.013 0.012
listcomp + func : 0.017 0.017 0.017 0.018 0.018
filter + lambda : 0.015 0.016 0.016 0.015 0.015
filter + func   : 0.024 0.015 0.014 0.014 0.015



### Listcomps For Cartesian Products
Listcomps can also be used to calulate the cartesian product of two or more iterables. The resulting list will have length equal to the no of iterables of both the lists multiplied
    
Lets see an example below:
    

In [9]:
colors = ['black','white']
sizes = ['s','M','L']

Now lets find the cartesian product using listcomp

In [10]:
tshirts = [(color,size) for color in colors
                        for size in sizes]
tshirts

[('black', 's'),
 ('black', 'M'),
 ('black', 'L'),
 ('white', 's'),
 ('white', 'M'),
 ('white', 'L')]

Here we see the no of elememts of the resulting list are the no of elements of both list multiplied. It is also recommended to indent the for loops below each other so the code can be more readable.
Here the items are first arrabged by colors and then sizes as you can see above.

The equivalent `for` loop for the same thing would be:

In [11]:
for color in colors:
    for size in sizes:
        print((color,size))

('black', 's')
('black', 'M')
('black', 'L')
('white', 's')
('white', 'M')
('white', 'L')


To arrange first by size and then by color just interchange the for loops

In [12]:
tshirts = [(color, size) for size in sizes
                         for color in colors]
tshirts

[('black', 's'),
 ('white', 's'),
 ('black', 'M'),
 ('white', 'M'),
 ('black', 'L'),
 ('white', 'L')]

Listcomps are useful for only building new lists. For building another type of sequences using generator expressions is the way to go.

### Generator Expressions
To initialize a tuple, array and other type of sequences you can use listcomp but genexp saves memory because it yields the items one by one using the iterator protocol rather than just bulding a whole new list to feed another constructor.

Genexp are same as listcomps instead of using `[ ]` "brackets" they use `( )` "parentheses"
Let's see an example below:

In [14]:
symbols = '$¢£¥€¤'
tuple((ord(symbol) for symbol in symbols))

(36, 162, 163, 165, 8364, 164)

If the generator expressions are the only single argument then there is no need for an enclosing parentheses.

But in case there are multiple arguments then an enclosing parentheses are mandatory. The first argument in the array constructor below is the type code for defining the type of values inside the array. In this case, its 'I' which stands for unsigned int

In [15]:
import array
array.array('I',(tuple((ord(symbol) for symbol in symbols))))

array('I', [36, 162, 163, 165, 8364, 164])

Now let's perform the cartesian product using genexp rather than listcomp as we did earlier. The advantage of using genexp is that the resulting list is never built-in memory like listcomp instead the genexp feeds the for loop items one by one. This is very effective when performing cartesian product on large list. 
It saves the expense of building list with millions of itmes inside it.

In [16]:
colors = ['black', 'white']
size = ['S', 'M', 'L']
for tshirt in ('%s %s' % (c, s) for c in colors 
                                for s in sizes):
    print(tshirt)

black s
black M
black L
white s
white M
white L


### Tuples Are Not Just Immutable Lists
Tuples are often described as just "immutable lists".But they can be also used as records without field names.

#### Tuples as Records
The way tuples can be used as a record is that each item in a tuple will hold value for one field and the position of the item defines the meaning of the field.
Sorting this tuples would destroy the data as the meaning of each item is represented by its position in the list.
See the example below

In [17]:
lax_coordinates = (33.9483, -118.325323)

Here the first position in the tuple represents the latitude and the second represents the longitude

In [18]:
city, year, pop, chg, area = ('Tokyo', 2003, 32540, 0.66, 8014)
city

'Tokyo'

In [19]:
year

2003

Here we assign the value to different fields using a well positioned tuple

In [21]:
traveler_ids = [('USA', '31195855'), ('BRA', 'CE342567'), ('ESP', 'XDA205856')]
for passport in sorted(traveler_ids):
    print('%s %s' % passport)

BRA CE342567
ESP XDA205856
USA 31195855


Here we print the values from the sorted list of tuples.  

In [23]:
for country,_ in sorted(traveler_ids):
    print(country)

BRA
ESP
USA


The for loop knows how to retrieve values from tuples this is known as tuple "unpacking". Since we want only the country name we assign a `'_'` dummy variable to the second field.

#### Tuple Unpacking

The most visible form of tuple unpacking is parallel assignment, which assigns items from an iterables to a tuple fo variables. See below

In [24]:
lax_coordinates = (33.9425, -118.408056)
latitude, longitude = lax_coordinates # tuple unpacking    
latitude

33.9425

In [25]:
longitude

-118.408056

Tuple unpacking also allows to swap variables without the ned of any temporary variable.

In [28]:
a, b = (2, 4)
a, b = b, a
a, b

(4, 2)

Another example of tuple unpacking is by prefixing a `*` when calling a function

In [29]:
divmod(20, 8)

(2, 4)

In [30]:
t = (20, 8)
divmod(*t)

(2, 4)

Here we unpack the tuple and provide them as arguments to the divmod function.

Let's see another example of tuple unpacking for extracting a filename from the path. For this we use `os.path.split()` which returns a `tuple(path, lastpart)` when provided a filepath.

In [31]:
import os
_, filename = os.path.split('/home/enigma/file.txt')
filename

'file.txt'

In [33]:
path = "/home/enigma/file.txt"
*_, filename = tuple(path.split('/'))
filename, _

('file.txt', ['', 'home', 'enigma'])

We can also split a string using the `split()` method to generate a tuple.
Using the prefix `*` you can see all the preceeding tuple elements are unpacked to the dummy variable and the last element is appropriately unpacked to the filename variable.

### Using `*` to grab excess items

The `*` prefix can be used to grab excess arguments provided to a function as well.


In [34]:
a, b, *rest = range(5)
a, b, rest

(0, 1, [2, 3, 4])

In [35]:
a, *rest, b = range(10)
a, b, rest

(0, 9, [1, 2, 3, 4, 5, 6, 7, 8])

The `*` prefix can only be applied to any one variable but it can appear in any position you want.

### Nested Tuple Unpacking
If the expression matches the nesting sructure python will conveniently unpack the variables to the respective variables.

In [37]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)), 
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

print('{:15} | {:^9} | {:^9}'.format('', 'lat', 'long'))
fmt = '{:15} | {:9.4f} | {:9.4f}'
for name, cc, pop, (latitude, longitude) in metro_areas:
    if longitude <= 0:
        print(fmt.format(name, latitude, longitude))

                |    lat    |   long   
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
Sao Paulo       |  -23.5478 |  -46.6358


As designed, tuples are very handy. But there is a missing feature when using them as records: sometimes it is desirable to name the fields. That is why the namedtuple function was invented. 

### Named Tuples
The `collections.namedtuple` function is a factory that produces subclass of tuple enhanced with class and field names. This also easier for debugging.


In [39]:
from collections import namedtuple
City = namedtuple('City', 'name country population coordinates')

Here we created a class named `City` with field names name, country, population and coordinates. The field names are provided by a space seperated string. 
Another equivalent approach is by providing a list for field names.

In [ ]:
City = namedtuple('City', []'name', 'country', 'population', 'coordinates'])

Lets create an object of the class `City`.

In [40]:
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
tokyo

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))

In [41]:
tokyo.population

36.933

In [42]:
tokyo.coordinates

(35.689722, 139.691667)

Since `namedtuple` allows to set field names we can extract information by using the field names instead of remembering at which position which field exists.

In [43]:
tokyo[1]

'JP'

Ofcourse, position still works well since it is a subclass of tuples.
Some of the other functionalities it inherits form the tuple superclass are:
1. `_fields` class attribute
2. `_make(iterable)` method
3. `_asdict()` instance method

In [44]:
City._fields

('name', 'country', 'population', 'coordinates')

In [45]:
Latlong = namedtuple('Latlong', 'lat long')

Let's use this newly created `namedtuple` with class name `'Latlong'` and fields `'lat'`and `'long'`. inside the `City` namedtuple class.

In [48]:
delhi_data = ('Delhi NCR', 'IN', 21.935, Latlong(28.613889, 77.208889))
delhi = City._make(delhi_data)

Here we created a tuple named `delhi_data` which is an iterable.
`_make()` allows you to instantiate a named tuple from an iterable; `City(*delhi_data)` would do the same.


In [49]:
delhi._asdict()

OrderedDict([('name', 'Delhi NCR'),
             ('country', 'IN'),
             ('population', 21.935),
             ('coordinates', Latlong(lat=28.613889, long=77.208889))])

In [50]:
for key, value in delhi._asdict().items():
    print(key + ':', value)

name: Delhi NCR
country: IN
population: 21.935
coordinates: Latlong(lat=28.613889, long=77.208889)


### Tuples As Immutable Lists
Now that we explored the use of tuples as records lets see their use as Immutable Lists

Tuple supports all list methods that do not involve adding or removing items, with one exception—tuple lacks the `__reversed__` method. However, that is just for optimization; `reversed(my_tuple)` works without it.

![title](assets/chp2-listvstuple.png)

### Slicing

All the sequences in python support the slicing feature. 

#### Why Slices and Range Exlude the Last Item?

Excluding last item works well with zero-based indexing.
It's easy to cinoute the length of slice or range by just subtracting start from stop.
It's easy to split the list without overlapping such as `a[x: ]` and `a[ :x]`

In [51]:
l = [10,20,30,40,50,60]
l[:3]

[10, 20, 30]

In [52]:
l[3:]

[40, 50, 60]

Another way of using slice is by mentioning the step as well.
`slice[start:stop:step]`
The step can be negative as well which will retrieve the items in reverse.

In [53]:
s = 'bicycle'
s[::3]


'bye'

In [54]:
s[::-1]

'elcycib'

In [55]:
s[::-2]

'eccb'

The notation a:b:c is only valid within `[]` when used as the indexing or subscript operator, and it produces a slice object: `slice(a, b, c)`.
To evaluate the expression `seq[start:stop:step]` python calls the seq.`__getitem__(slice(start,stop,step)`.
Knowing about slice objects is useful because it lets you assign name to slice objects.

Suppose you need to parse flat-file data like the invoice shown below. Instead
of filling your code with hardcoded slices, you can name them. See how readable this makes the for loop at the end of the example.

In [66]:
invoice = """
0.....6.................................40........52...55........
1909  Pimoroni PiBrella                     $17.50    3    $52.50
1489  6mm Tactile Switch x20                 $4.95    2     $9.90
1510  Panavise Jr. - PV-201                 $28.00    1    $28.00
1601  PiTFT Mini Kit 320x240                $34.95    1    $34.95
"""
SKU = slice(0, 6)
DESCRIPTION = slice(6, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL=slice(55, None)
line_items = invoice.split('\n')[2:]
for item in line_items:
    print(item[UNIT_PRICE], item[DESCRIPTION])

    $17.50   Pimoroni PiBrella                 
     $4.95   6mm Tactile Switch x20            
    $28.00   Panavise Jr. - PV-201             
    $34.95   PiTFT Mini Kit 320x240            
 


### Multidimensional Slicing 
The `[]` operator also supports multiple slice objects seperated by comma(,). Though all the built-in sequences in python are one dimensional. This is particularly useful while dealing with two dimensional or n dimensional array slicing using the numpy package.
For example, in case of two dimensional array slicing can be applied as `a[m:n,i:j]`. 
The `__getitem__` and `__setitem__` special methods that handle the `[]` operator simply receive the indices in `a[i, j]` as a tuple. In other words, to evaluate `a[i, j]`, Python calls `a.__setitem__((i, j))`.

### Assigning to Slices
Mutable sequences can be altered in place using slice notation on the left side of the assignment operator or as the target of the del statement.

In [67]:
l = list(range(10))
l

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [68]:
l[2:5] = [20,30]
l

[0, 1, 20, 30, 5, 6, 7, 8, 9]

In [69]:
del l[5:7]
l

[0, 1, 20, 30, 5, 8, 9]

In [71]:
l[3::2] = [11, 22]
l

[0, 1, 20, 11, 5, 22, 9]

In [72]:
l[2:5] = 100

TypeError: can only assign an iterable

When the target of an assignment is a slice the right side must be an iterable object

### Using `+` and `*` with sequences
Usually when using `+` both the operands must be of the same sequence type. Neither of them is modified rather python creates a new sequence.

In [73]:
l = [1,2,3]
id(l)

58231504

In [74]:
l*3

[1, 2, 3, 1, 2, 3, 1, 2, 3]

In [75]:
id(l*3)

57954256

Here you can see the id of both the list is different. Both `+` and `*` create a new sequence and never change their operands.

### Building lists of lists
When we need to initialize a number of nested lists the best way of doing so is using listcomps.

In [76]:
board = [[' '] * 3 for i in range(3)]
board

[[' ', ' ', ' '], [' ', ' ', ' '], [' ', ' ', ' ']]

In [78]:
board[2][1] = 'x'
board

[[' ', ' ', ' '], [' ', ' ', ' '], [' ', 'x', ' ']]

A common mistake performed by programmers as a shortcut is:

In [79]:
weird_board = [[' '] * 3] * 3
weird_board

[[' ', ' ', ' '], [' ', ' ', ' '], [' ', ' ', ' ']]

Everything seems alright. Right?

In [81]:
weird_board[1][2] = 'x'
weird_board

[[' ', ' ', 'x'], [' ', ' ', 'x'], [' ', ' ', 'x']]

Using `*` to create nested lists actually create same references to the inner nested list instead of creating a new one.
It behaves like this code:

In [82]:
row = ['_'] * 3
board = []
for i in range(3):
    board.append(row)

The same row is appended each time to the board.
Whereas, using the listcomp above to build nested lists performs like this:

In [85]:
board = []
for i in range(3):
    row  = [ ] * 3
    board.append(row)

On each iteration of the for loop a new row is created which is then appended to the board.

### Augmented Assignment Operators
The augmented assignment operators `+=` and `*=` behave very differently depending on the first operand. 
The special method that makes `+=` work is `__iadd__` (for “in-place addition”). However, if `__iadd__` is not implemented, Python falls back to calling `__add__`. Consider this simple expression:

In [86]:
a += b

Firstly, if a implements `__iadd__`, that will be called. Then if a belongs to mutable sequences(`list`, `bytearray`, `array.array`) then a will be changed in place and will be similar to using `a.extend(b)`. However, if neither of those are implemented python fallbacks to using `__add__` and the operation will be similar to using `a = a + b`.
In general, for mutable sequences, it is a good bet that `__iadd__` is implemented and that `+=` happens in place. For immutable sequences, clearly there is no way for that to happen.This also applies to `*=`, which is implemented via `__imul__`. 

In [87]:
l = [1,2,3]
id(l)

58159272

In [88]:
l *= 2
id(l)

58159272

As you can see when we perform `+=` or `*=` operators on mutable sequences the object remains the same instead new items are appended.

In [89]:
t = (1, 2, 3)
id(t)

57354240

In [91]:
t *= 2
id(t)

57333440

Unlike in the case of mutable sequences, tuples are immutable so the operator `+=` and `*=` altogether create a new tuple. 

Repeated concatenation of immutable sequences is inefficient, because instead of just appending new items, the interpreter has to copy the whole target sequence to create a new one with the new items concatenated.

In [92]:
t = (1, 2, [30,40])
t[2] += [50,60]

TypeError: 'tuple' object does not support item assignment

As expected this throws an error as assignment is not supported in tuples.
But let's try outputting the tuple

In [93]:
t

(1, 2, [30, 40, 50, 60])

The list inside the tuple was modified although it throwed an error.
This example is quite a corner case.

There are three lessons to be taken from this:
1. Inserting mutable items inside tuples is not a good idea.
2. Augmented assignment is not an atomic operation as it completed a part of its operation before throwing error.
3. Inspecting python bytecode is useful for debugging.

### `List.sort` and `Sorted` Built-In Function
The `list.sort` method sorts a list in place without creating a new one or making a copy. If it returns `None` to remind that the target object is changed and a new list is not created.

This is important part of Python API convention: functions or methods change the object in place and should return None to make clear that no new object was created.

The built-in sorted function creates a new list and returns it. Also it accepts any iterable object as an argument, including generators and immutable sequences. If it is of the type iterable then it always returns a new list.

Both the functions take two arguments:
1. `reverse` - if True items are returned in descending order.
2. `key` - A one argument function that will be applied to each item to produce its sorting key. 
    eg, when sorting strings , `key = str.lower` will perform a case-insensitive sort. `key = len` will sort on the bais of length of strings.

In [96]:
fruits = ['Apple', 'Mango', 'Pineapple', 'Guava']
sorted(fruits)

['Apple', 'Guava', 'Mango', 'Pineapple']

This sorts the list alphabetically.

In [97]:
sorted(fruits, reverse=True)

['Pineapple', 'Mango', 'Guava', 'Apple']

This sorts the list and returns in reverse order.

In [98]:
sorted(fruits, key=len)

['Apple', 'Mango', 'Guava', 'Pineapple']

This sorts the string on the basis of the string length

In [99]:
fruits.sort()
fruits

['Apple', 'Guava', 'Mango', 'Pineapple']

This sorts the target list and doesn't create new one.
Since it returns None we need to mention the list to print the list.

### Managing Ordered Sequences with bisect
Sorted sequences are efficient to be searched on.
The Python Standard Library provides a bisect module for binary searching.

The two main functions are `bisect` and `insort`.

`Bisect(lista, b)` does a binary search to locate where b can be stored in the list such that the list remains sorted. The `index` returned by `bisect` can be used to insert the value at right place in the list.

Using `insort` is both faster and better than this.

In [126]:
import bisect,sys
HAYSTACK = [1, 4, 5, 6, 8, 12, 15, 20, 21, 23, 23, 26, 29, 30]
NEEDLES = [0, 1, 2, 5, 8, 10, 22, 23, 29, 30, 31]
ROW_FMT = '{0:2d} @ {1:2d}    {2}{0:<2d}'

def demo(bisect_fn):
    for needle in reversed(NEEDLES):
        position = bisect_fn(HAYSTACK, needle)
        offset = position * '  |'
        print(ROW_FMT.format(needle, position, offset))
        
if __name__ == '__main__':
    
    if sys.argv[-1] == 'left':
        bisect_fn = bisect.bisect_left
    else:
        bisect_fn = bisect.bisect

    print('DEMO: ', bisect_fn.__name__)
    print('haystack ->', ' '.join('%2d' % n for n in HAYSTACK))
    demo(bisect_fn)    

DEMO:  bisect_right
haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30
31 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |31
30 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |30
29 @ 13      |  |  |  |  |  |  |  |  |  |  |  |  |29
23 @ 11      |  |  |  |  |  |  |  |  |  |  |23
22 @  9      |  |  |  |  |  |  |  |  |22
10 @  5      |  |  |  |  |10
 8 @  5      |  |  |  |  |8 
 5 @  3      |  |  |5 
 2 @  1      |2 
 1 @  1      |1 
 0 @  0    0 


The behavior of bisect can be fine-tuned in two ways.

First, a pair of optional arguments, `lo` and `hi`, allow narrowing the region in the sequence to be searched when inserting. `lo` defaults to `0` and `hi` to the `len()` of the sequence.

Second, `bisect` is actually an alias for `bisect_right`, and there is a sister function called `bisect_left`. Their difference is apparent only when the needle compares equal to an item in the `list: bisect_right` returns an insertion point after the existing item, and `bisect_left` returns the position of the existing item, so insertion would occur before it.

Bisect can also be used for table lookups by numeric values such as converting scores to grades.

In [127]:
def grade(score, breakpoints = [60, 70, 80, 90], grades = 'FDCBA'):
    i = bisect.bisect(breakpoints, score)
    return grades[i]

[grade(score) for score in [33, 99, 77, 70, 89, 90, 100]]

['F', 'A', 'C', 'C', 'B', 'A', 'A']

### Inserting with `bisect.insort` 
Sorting is expensive, so once you have a sorted sequence, it’s good to keep it that way. That is why `bisect.insort` was created. `insort(seq, item)` inserts item into seq so as to keep seq in ascending order. 


In [129]:
import bisect
import random

SIZE = 7
random.seed(2048)

my_list = []
for i in range(SIZE):
    new_item = random.randrange(SIZE * 2)
    bisect.insort(my_list, new_item)
    print('%2d ->' % new_item, my_list)

 4 -> [4]
 7 -> [4, 7]
 8 -> [4, 7, 8]
 6 -> [4, 6, 7, 8]
11 -> [4, 6, 7, 8, 11]
 8 -> [4, 6, 7, 8, 8, 11]
10 -> [4, 6, 7, 8, 8, 10, 11]


`Bisect.insort` also takes optional arguments `lo` and `hi` to narrow the search to a sub sequence.
Bisect applies to all types of sequences in python in genral not just list and tuples. Dont overuse lists ifyou are using a list of numbers then array is the way to go.
Lists are flexible and easy, but, depending on the situation other sequences can be used.
For example arrays can be used to store large quantities of floating point numbers as they store the packed bytes and not the whole float object.
If the situation demands removing elements from the ends of sequence such as LIFO OR FIFO then using `deque` or double ended queue works faster.

### Arrays

Arrays are more efficient ata storing numbers than lists.They support all the utable sequences operations and some additional operations for fast loading and saving such as `.frombytes` and `.tobytes`.

A Python array is as lean as a C array. When creating an array, you provide a typecode, a letter to determine the underlying C type used to store each item in the array. For example, `b` is the typecode for `signed char`. If you create an `array('b')`, then each item will be stored in a single byte and interpreted as an integer from `–128` to `127`. For large sequences of numbers, this saves a lot of memory. And Python will not let you put any number that does not match the type for the array.

Here is a list of typecodes in arrays:
![title](assets/chp2-arraytypecodes.png)

In [132]:
from array import array
from random import random
floats = array('d',(random() for i in range(10**7)))
floats[-1]

0.9720496238128803

In [133]:
fp = open('floats.bin','wb')
floats.tofile(fp)
fp.close()

This writes the above array to a file named **"floats.bin"**

In [135]:
floats2 = array('d')
fp = open('floats.bin', 'rb')
floats2.fromfile(fp,10**7)
fp.close()
floats2[-1]

0.9720496238128803

In [136]:
floats2 == floats

True

After importing the exported float array from the `"floats.bin"` to another array, they both match.
`Array.tofile` and `Array.fromfile` are not only easy to use but they are also super fast. It takes 0.1s to load 10 million double-precision floats from file created using `Array.tofile`. This is nearly 60 times faster than reading the numbers from a text file, which also involves parsing each line with the float built-in. 

Saving with `array.tofile` is about 7 times faster than writing one float per line in a text file. In addition, the size of the binary file with 10 million doubles is 80,000,000 bytes (8 bytes per double, zero overhead), while the text file has 181,515,739 bytes, for the same data.

##### Tip
 
 Another fast and more flexible way of saving numeric data is the
 `"pickle"` module for object serialization. Saving an array of floats
 with pickle.dump is almost as fast as with `array.tofile`—how‐
 ever, pickle handles almost all built-in types, including complex
 numbers, nested collections, and even instances of user-defined
 classes automatically (if they are not too tricky in their implemen‐
 tation).
    
Here are all the methods and attributes found in list and array:
![title](assets/chp2-listvsarray.png)

#### Memory Views
The built-in memorview class is a shared-memory sequence type that lets you handle slices of arrays without copying bytes.

Using notation similar to the array module, the `memoryview.cast` method lets you change the way multiple bytes are read or written as units without moving bits around (just like the C cast operator). `memoryview.cast` returns yet another memoryviewobject, always sharing the same memory.

In [152]:
import array
numbers = array.array('h', [-2, -1, 0, 1, 2])
memv = memoryview(numbers)
len(memv)

5

Here we created a memv from and array of short signed int 'h'. Infactthe memoryview sees the same 5 items as you can see below.

In [153]:
memv[0]

-2

Let's cast the short signed integers to unsigned char and output it as a list. Each signed int is output as 2 bytes of unsigned char values. 

In [155]:
memv_oct = memv.cast('B')
memv_oct.tolist()

[254, 255, 255, 255, 0, 0, 1, 0, 2, 0]

In [156]:
memv_oct[5] = 4
numbers

array('h', [-2, -1, 1024, 1, 2])

Note change to numbers: a 4 in the most significant byte of a 2-byte unsigned
integer is `1024`.

### Deques and Other Queues
The `.append` and `.pop` methods make a list usable as a stack or a queue (if you 
use `.append` and `.pop(0)`, you get LIFO behavior). But inserting and removing from the left of a list (the 0-index end) is costly because the entire list must be shifted.

The class `collections.deque` is a thread-safe double-ended queue designed for fast inserting and removing from both ends. It is also the way to go if you need to keep a list of “last seen items” or something like that, because a deque can be bounded—i.e., created with a maximum length—and then, when it is full, it discards items from the opposite end when you append new ones.

In [172]:
from collections import deque
dq = deque(range(10), maxlen=10)
dq

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

The optional `maxlen` argument sets the maximum number of items allowed in this instance of deque; this sets a read-only maxlen instance attribute.

In [173]:
dq.rotate(3)
dq

deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6])

In [174]:
dq.rotate(-4)
dq

deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])

Rotating with `n > 0` takes items from the right end and prepends them to the left; when `n < 0` items are taken from left and appended to the right.

In [175]:
dq.appendleft(-1)
dq

deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9])

Appending to a deque that is full `(len(d) == d.maxlen)` discards items from
the other end; note in the next line that the `0` is dropped.

In [176]:
dq.extend([11, 22, 33])
dq

deque([3, 4, 5, 6, 7, 8, 9, 11, 22, 33])

In [177]:
dq.extendleft([10, 20, 30, 40])
dq

deque([40, 30, 20, 10, 3, 4, 5, 6, 7, 8])

Adding three items to the right pushes out the leftmost `-1`, `1`, and `2`.

Note that `extendleft(iter)` works by appending each successive item of the iter argument to the left of the deque, therefore the final position of the items is reversed.

Note that deque implements most of the list methods, and adds a few specific to its design, like `popleft` and `rotate`. But there is a hidden cost: removing items from the middle of a deque is not as fast. It is really optimized for appending and popping from the ends.

The `append` and `popleft` operations are atomic, so deque is safe to use as a LIFO queue in multithreaded applications without the need for using locks.
![title](assets/chp2-listvsdeque.png)